In [24]:
import os
import cv2 as cv
from matplotlib import pyplot
import numpy as np

%matplotlib inline
pyplot.rcParams['figure.figsize'] = [24, 10]

os.chdir("F:/Pi13C/to analyze")

fileList = os.listdir("./")
fileList = [f for f in fileList if not f.endswith("TRUTH.png")]
fileList = [f for f in fileList if not f.endswith("ANALYZED.png")]
fileList = [f[:-4] for f in fileList]

originalImages = [cv.imread(f + ".png") for f in fileList]
analyzedImages = [cv.threshold(cv.imread(f + "_ANALYZED.png"), 0, 255, cv.THRESH_BINARY)[1] for f in fileList]
truthImages = [cv.threshold(cv.imread(f + "_TRUTH.png"), 0, 255, cv.THRESH_BINARY)[1] for f in fileList]

assert len(originalImages) == len(analyzedImages)
assert len(analyzedImages) == len(truthImages)

In [31]:
truePositiveCounts = [float(np.sum(i/255)) for i in truthImages]
positiveCounts = [float(np.sum(i/255)) for i in analyzedImages]
truePositiveRatios = [truePositiveCounts[i]/positiveCounts[i] for i in range(len(truePositiveCounts))]
print("mean true positive ratio: " + str(np.sum(truePositiveRatios)/len(truePositiveRatios)))

mean true positive ratio: 0.9808395058534464
